In [2]:
from IPython.display import Image
import pandas
import numpy
import matplotlib
import matplotlib.pyplot as plt 
%matplotlib inline

plt.rcParams["figure.figsize"]=[10,10]
numpy.random.seed(42)

In [3]:
#Optimizacion de hiperparamentros (aquellos parametros que no estan dentro del modelo en si, pero afectan su rendimiento)

#Hasta ahora hemos visto una manera relativamente sencilla de ver que valores de los hiperparametros funcionan mejor, mediante las curvas de validacion
#Estas curvas son muy utiles para darnos informacion a los Data Scientits, pero tienen dos problemas:
#1. son metodos graficos, esto significa que necesitan un humano para interpretarlas y no nos permiten automatizar el proceso para encontrar los hiperparametros optimos
#2. solo toman un hiperparametro a la vez. Esto significa que hacen que sea mas dificil el evaluar combinaciones de los hiperparametros (si quisieramos evaluar multiples hiperparametros tendriamos que hacer graficas de planos o hiperplanos)

#Se veran metodos mas robustos para dado un modelo, encontrar el conjunto de hiperparametros que hace que funcion mejor

In [4]:
data=pandas.read_csv(r"D:\Estudiar\MLData\boston\Boston.csv")

In [5]:
data["medv"]=data["medv"]>21

In [6]:
sum(data["medv"])

257

In [7]:
data.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,True
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,True
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,True
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,True
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,True


In [8]:
#Pipeline de procesamiento de datos

In [9]:
targetN="medv"
trainN=data.drop(targetN,axis=1).columns
train=data[trainN]
target=data[targetN]

In [10]:
target.unique()

array([ True, False])

In [11]:
data_num=train.select_dtypes([int, float])
col_num=data_num.columns

data_cat=train.select_dtypes([object])
col_cat=data_cat.columns

In [12]:
#Se usara un trasformados MultiLabelBinarizer. Es como el LabelBinarizer pero funciona para multiples columnas

In [13]:
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer
from sklearn.impute import SimpleImputer

In [14]:
b=MultiLabelBinarizer()
b.fit_transform(
    [
        ["gato","patata","rojo"],
        ["perror","zanahoria","axul"],
        ["camello","patata","verde"],
        ["gato","patata","rojo"]
    ]
)

array([[0, 0, 1, 1, 0, 1, 0, 0],
       [1, 0, 0, 0, 1, 0, 0, 1],
       [0, 1, 0, 1, 0, 0, 1, 0],
       [0, 0, 1, 1, 0, 1, 0, 0]])

In [15]:
b.classes_

array(['axul', 'camello', 'gato', 'patata', 'perror', 'rojo', 'verde',
       'zanahoria'], dtype=object)

In [76]:
from sklearn.pipeline import Pipeline, FeatureUnion

In [77]:
from sklearn.base import TransformerMixin
class ColumnExtractor(TransformerMixin):
    def __init__(self,columns):
        self.columns=columns

    def transform(self, X, **tranform_params):
        return pandas.DataFrame(X[self.columns])

    def fit(self, X, **fit_params):
        return self

class BinarizadorMultipleCat(TransformerMixin):
    def __init__(self,*args, **kargs):
        self.encoder=MultiLabelBinarizer(*args, **kargs)
    def fit(self, X, y=None):
        #super(BinarizadorMultipleCat,self).fit(X)
        self.encoder.fit(X)
        return self

    def transform(self, X, y=None):
        return self.encoder.transform(X)


In [78]:
pipeline_num=Pipeline([
    ("selecct_num",ColumnExtractor(columns=col_num)),
    ("escalador",StandardScaler()),
    ("imputador",SimpleImputer()),
])

pipeline_cat=Pipeline([
   ("selector_cat",ColumnExtractor(columns=col_cat)),
  ("codificador_numerico",BinarizadorMultipleCat()),
])

In [79]:
pipeline_cat.fit_transform(train)

array([], shape=(0, 0), dtype=int32)

In [80]:
pipeline_num.fit_transform(train).shape

(506, 10)

In [90]:
pipeline_procesado=FeatureUnion([
    ("trans_num",pipeline_num),
    #("trans_cat",pipeline_cat),
])

In [91]:
train_pro=pipeline_num.fit_transform(train)

In [83]:
train_pro.shape

(506, 10)

In [24]:
from sklearn.model_selection import cross_validate

def evaluar_modelo(estimador, X, y):
    res_estimador=cross_validate(estimador, X, y, scoring="roc_auc",n_jobs=2, cv=5, return_train_score=True)
    return res_estimador

def see_res():
    res_df=pandas.DataFrame(resultados).T
    res_cols=res_df.columns
    for col in res_df:
        res_df[col]=res_df[col].apply(numpy.mean)
        res_df[str(col)+"_idx"]=res_df[col]/res_df[col].max()
    return res_df


In [25]:
from sklearn.linear_model import LogisticRegression 
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
resultados={}

In [26]:
resultados["reg_logistica"]=evaluar_modelo(LogisticRegression(),train_pro,target)
resultados["naive_bayes"]=evaluar_modelo(GaussianNB(),train_pro,target)
resultados["rf"]=evaluar_modelo(RandomForestClassifier(),train_pro,target)
resultados["svc"]=evaluar_modelo(SVC(),train_pro,target)

In [27]:
see_res()

,fit_time,score_time,test_score,train_score,fit_time_idx,score_time_idx,test_score_idx,train_score_idx
reg_logistica,0.014928,0.002974,0.914224,0.948886,0.059689,0.180356,0.987511,0.948890
naive_bayes,0.003559,0.000000,0.865827,0.883308,0.014232,0.000000,0.935234,0.883313
rf,0.250090,0.016487,0.914403,0.999995,1.000000,1.000000,0.987705,1.000000
svc,0.012101,0.008097,0.925786,0.970662,0.048388,0.491092,1.000000,0.970667


In [28]:
#Se selecciona un estimador en funcion de los resultados iniciales y optimizarlo. Elijo el estimador Random Forest por que funciona muy bien en comparacion a los demas y es bastante rapido de entrenar

In [29]:
estimador_rf=RandomForestClassifier()

In [30]:
#Scikit-learn tiene dos metodos de optimizacion de hiperparametros, GridSearchCV y RandomizedSearchCV

#GridSearchCV funciona realizando una busqueda en una malla, es decir, pasandole un conjunto de posibles opciones de hiperparametros evalue de forma completa cada combinacion de dichos parametros (es decir, el valor 1 del hiperparametro 1 combinado con todos los posibles valores de los demas hiperparametros, el valor 2 del hiperparametros 1 combinado con todos los posibles valores de los demas hiperparametros, etcetera)

#La ventaja de utilizar una busqueda de malla es que nos aseguramos de que se han probado todas las combinaciones posibles. El problema es que el proceso requiere mucho tiempo de computacion, y segun que dataset usemos

In [31]:
%%timeit
import time
def foo():
    time.sleep(1)

360 ns ± 57.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [32]:
%%timeit -n 1 -r 1 #n ejecuta esta celda una vez, r que ejecute un solo loop
def foo():
    time.sleep(1)

600 ns ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [35]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [36]:
estimador_rf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [42]:
numpy.linspace(10,1000,10).astype(int)

array([  10,  120,  230,  340,  450,  560,  670,  780,  890, 1000])

In [43]:
param_search_rf={
    "criterion": ["gini","entropy"],
    "n_estimators": numpy.linspace(10,1000,10).astype(int),
    "class_weight": [None,"balanced"]
}

grid=GridSearchCV(estimator=estimador_rf, param_grid=param_search_rf, scoring="roc_auc",n_jobs=2)

In [44]:
#GridSearchCV se comporta como un estimador en cuanto a que tienen un metodo fit que usamos para "entrenarlo" y que realize la busqueda en malla
#Para ver cuanto tiempo tarda en realizar la busqueda usamos %%timeit que evalua el tiempo que tarda una funcion en ejecutarse

In [45]:
%%timeit -n 1 -r 1
grid.fit(train_pro,target)

2min 41s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [46]:
print(grid.best_score_)
print(grid.best_estimator_)

0.9212749715270725
RandomForestClassifier(class_weight='balanced', criterion='entropy',
                       n_estimators=560)


In [47]:
#Tras haberlo ajustado, Gridsearch nos devuelve el ranking de todas las variantes evaluadas junto con metricas de su funcionamiento con el atributo cv_results_

In [49]:
pandas.DataFrame(grid.cv_results_).sort_values(by="rank_test_score");

In [50]:
resultados["rf_gridsearch"]=evaluar_modelo(grid.best_estimator_, train_pro, target)

In [51]:
#Se hara la misma optimizacion de parametros pero usando RandomizedSearchcCV, el cual funciona de forma similar a GridSearchCV, pero en vez de evaluar todas las combinaciones posibles de hiperparametros, se toman n muestras de hiperparametros de dichas distribuciones en vez de valores fijos para hiperparametros continuos

#Primero se va a evaluar el funcionamiento de la busqueda aleatoria con los mismos hiperparametros que hemos usado en la busqueda en malla. para "RandomizedSearchCV" tenemos que indicarle cuantas variables de hiperparametros utilizar (definidas por el parametro n_iter, por defecto toma 10 variantes). Dado que dicha busqueda tom amuestreos el parametro ya no se llama "param_grid" sino "param_distributions"

In [54]:
busqueda_ran=RandomizedSearchCV(estimator=estimador_rf, param_distributions=param_search_rf, scoring="roc_auc", n_jobs=2, n_iter=10)

In [55]:
%%timeit -n 1 -r 1
busqueda_ran.fit(train_pro,target)

49.1 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [56]:
print(busqueda_ran.best_score_)
print(busqueda_ran.best_estimator_)

0.9190854649552127
RandomForestClassifier(class_weight='balanced', criterion='entropy',
                       n_estimators=450)


In [57]:
#La busqueda de malla obtivo 0.9212749715270725 contra 0.9190854649552127obtenido por la busqueda aleatoria 
#sin embargo la busqueda aleatoria ha tardado 8 veces menos

In [58]:
resultados["rf_randomizedsearch"]=evaluar_modelo(grid.best_estimator_,train_pro, target)

In [59]:
#La ventaja del Randomized Search es que nos permite evaluar un espacio de hiperparametros mas amplio para un tiempo de computacion similar
#Paara ver esto se ampliara el espacio de busqueda de hiperparametros y hacer 20 muestreos

In [61]:
from scipy.stats import randint 

param_dist_ran={
    "max_depth": [3, None],
    "max_features": randint(1,11),
    "min_samples_split": randint(2,11),
    "min_samples_leaf": randint(1,11),
    "bootstrap": [True, False],
    "criterion": ["gini", "entropy"],
    "n_estimators": numpy.linspace(10,1000,10).astype(int),
}

In [62]:
busqueda_ran_100=RandomizedSearchCV(estimator=estimador_rf, param_distributions=param_dist_ran, scoring="roc_auc", n_jobs=2, n_iter=20)

In [63]:
%%timeit -n 1 -r 1
busqueda_ran_100.fit(train_pro,target)

1min 19s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [64]:
print(busqueda_ran_100.best_score_)
print(busqueda_ran_100.best_estimator_)

0.9283126173546341
RandomForestClassifier(max_depth=3, max_features=5, min_samples_leaf=2,
                       min_samples_split=7, n_estimators=450)


In [65]:
#La busqueda aleatoria con los nuevos parametros ha tardado un tiempo similar a la busqueda en malla. pero ha obtenido una puntuacion maxima ROC AUC de 0.9283126173546341

In [66]:
resultados["rf_randomizedsearch_100"]=evaluar_modelo(busqueda_ran_100.best_estimator_, train_pro, target)

In [67]:
see_res()

,fit_time,score_time,test_score,train_score,fit_time_idx,score_time_idx,test_score_idx,train_score_idx
reg_logistica,0.014928,0.002974,0.914224,0.948886,0.008308,0.023385,0.987511,0.948886
naive_bayes,0.003559,0.000000,0.865827,0.883308,0.001981,0.000000,0.935234,0.883308
rf,0.250090,0.016487,0.914403,0.999995,0.139186,0.129661,0.987705,0.999995
svc,0.012101,0.008097,0.925786,0.970662,0.006735,0.063675,1.000000,0.970662
rf_gridsearch,1.759247,0.127154,0.917864,1.000000,0.979098,1.000000,0.991443,1.000000
rf_randomizedsearch,1.796803,0.115709,0.918571,1.000000,1.000000,0.909992,0.992206,1.000000
rf_randomizedsearch_100,1.446913,0.106798,0.922525,0.973737,0.805271,0.839908,0.996477,0.973737


In [68]:
#PArace que el estimador obtenido con la busqueda aleatoria es de los que mejor funciona

#En general, salvo que el espacio de hiperparametros que queramos explorar sea pequeño, es mejor utilizar RandomizedSearch en ver de GridSearchCV. Esto es asi porque en general no existe un unico conjunto de hiperparametros que obtiene el mejor funcionamiento, sino que suelen existir multiples "areas" en el espacio dimensional de los hiperparametros que funcionan de forma similar. Al hacer una busqueda aleatoria podemos explorarlas las diversas areas en un tiempo mas reducido

In [69]:
#Optimizacion de parametros dentro de Pipeline

#Los algoritmos de busqueda de sklearn siguen la API de transformaciones y estimadores. Esto significa que podemos crear un pipeline e incluir la optimizacion de hiperparametros dentro

In [96]:
busqueda_ran_10=RandomizedSearchCV(estimator=estimador_rf, param_distributions=param_dist_ran, scoring="roc_auc", n_iter=10)

pipeline_estimador=Pipeline(
    [
        ("procesado", pipeline_num),
        ("estimador", busqueda_ran_10),
    ]
)

In [97]:
#ahora podemos ajustar directamente en los datos originales sin tener que preprocesarlos

In [99]:
pipeline_estimador.fit(train, target)

TypeError: fit() takes 2 positional arguments but 3 were given

In [100]:
#Scikit-optimize es una libreria que implementa multiples metodos para optimizar hiperparametros de forma secuencial

#desde jupyter se instala como:

In [110]:
import sys
!{sys.executable} -m pip install scikit-optimize

In [111]:
import skopt

In [112]:
skopt.__version__

'0.8.1'

In [113]:
from skopt import gp_minimize

In [114]:
#En vez de usar un diccionario con el espacio de hiperparametros que queremos buscar, scikit-optimize necesita pasarle una lista de parametros

#skopt es una libreria relativamente nueva, y tienen ciertas limitaciones comparada con scikit-optimize. Por ejemplo, en vez de diccionarios con los nombre con los parametros, espera como inputs listas, no se pueden usar funciones de distribuciones como scipy.randint

In [120]:
from skopt import space

param_espace_skopt=[
     space.Integer(3,10), #max_depth
     space.Integer(1,11), #max_features
     (0.001, 0.99, "uniform"), #min_samples_split
     (0.001, 0.5, "uniform"), #min_samples_leaf
     space.Integer(1, 1000), #n_estimators
     space.Categorical(["gini","rntropy"]), #criterion
     space.Categorical([True,False])
 ]

In [122]:
#scikit-optimeze necesita que definamos la funcion objetivo, que ira variando en funcion de los parametros elegidos. Dicha funcion tiene que crear el estimador y evaluarlo y devolver la evaluacion

In [130]:
from sklearn.model_selection import cross_val_score

estimador_rf=RandomForestClassifier()

def funcion_optimizaable(params):
    #params es una seleccion especifica de hiperparametros
    max_depth, max_features, min_samples_split, min_samples_leaf, n_estimators, criterion, bootstrap=params

    estimador_rf.set_params(
        max_depth=max_depth,
        max_features=max_features,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        n_estimators=n_estimators,
        criterion=criterion
    )

    return -numpy.mean(cross_val_score(estimador_rf, train_pro, target, cv=5, n_jobs=2, scoring="roc_auc"))

In [131]:
#ahora podemos dejar que skopt optimize los outputs de la funcion funcion_optimizable mediante el uso de gp_minimize

In [ ]:
%%timeit -n 1 -r 1
resultado_gp=gp_minimize(funcion_optimizaable, param_espace_skopt, n_calls=20, random_state=42)

In [ ]:
#Hyperopt-sklearn es una libreria que trabaja con scikit-learn